In [1]:
import os 
import glob
import numpy as np
import rasterio as rio
import pandas as pd
import json
import pyproj
from rasterio import plot
from rasterio.mask import raster_geometry_mask
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform
import geopandas as gpd

In [2]:
# Try clipping using a multipolygon geojson

def clip_to_multi_aoi(mask_path, rast_path):
    
    '''This function clips images to a multipolygon area of interest.
    
    Inputs:
    mask_path: path to a geojson file for the aoi (can be a multipolygon geometry)
    rast_path: path to the image to clip
    
    Output: saves the clipped image as new .tif file'''
    
    # Read aoi mask and update crs to match image 
    with rio.open(rast_path) as src:
        geo = gpd.read_file(mask_path)
        geo = geo.to_crs(src.crs)
    
        # Mask pixels outside the aoi
        # For more on the rasterio mask function: https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html
        
        out_image, out_transform = rio.mask.mask(src, geo.geometry, filled = True)
        #show(out_image)
        
        # Update output image metadata to match input image metadata
        mask_meta = src.meta
        mask_meta.update({"driver": "GTiff",
                          "height": out_image.shape[1],
                          "width": out_image.shape[2],
                          "transform": out_transform})
        
        # Change this depending on preferred output location
        with rio.open('/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2020/SkySat/'+
                      str(os.path.basename(rast_path).split('.tif')[0])+
                      "_clipped.tif", 
                      "w", 
                      **mask_meta) as dest:
            dest.write(out_image)  